## Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

## Creating CNN Model


In [ ]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


## Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data


## Loading the Data

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'users/PrajyotPatil/project/data_train.csv',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'users/PrajyotPatil/project/data_validate.csv',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


## Training the model

In [ ]:
model = create_model()

history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Depends on your dataset size
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50  # Depends on your validation dataset size
)


## Integrating with OpenCV

In [ ]:
def predict_weld_quality(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0  # Rescaling
    img = np.expand_dims(img, axis=0)  # Expanding dimensions to fit model's input
    prediction = model.predict(img)
    if prediction[0] > 0.5:
        print("Welded Properly")
    else:
        print("Not Welded Properly")
